In [25]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [65]:
# global variables
features = ["Age", "Gender", "Department"] # to be determined
target = "Attrition"


In [27]:
def main():
    # create dataframe with project's data set
    df = pd.read_csv('HR.csv')
    # clean the data
    clean_df = clean_data(df)
    # get features matrix X and target variable y
    X, y = feature_selection(clean_df)
    

In [72]:
def extract_categorical(df):
    # extract a list of all categorical features for feature engineering
    return df.select_dtypes(include=['object']).columns.tolist()

In [68]:
def clean_data(df):
    # drop columns that are not needed
    for col_header in df.columns:
        if col_header not in features and col_header != target:
            df.drop(col_header, axis = 1, inplace=True)         
    # drop rows that contain at least one empty cell
    df.dropna(axis='rows')    
    # feature enigneering for categorical data
    df = pd.get_dummies(data=df, columns=extract_categorical(df))   
    return df


In [33]:
def feature_selection(df):
    # split dataset in features and target variable
    X = df[features] # Features Matrix
    y = df[target] # Target variable
    return X, y


In [71]:
# test
df = pd.read_csv('HR.csv')
print(clean_data(df))



      Age  Attrition_No  Attrition_Yes  Department_Human Resources  \
0      41             0              1                           0   
1      49             1              0                           0   
2      37             0              1                           0   
3      33             1              0                           0   
4      27             1              0                           0   
...   ...           ...            ...                         ...   
1465   36             1              0                           0   
1466   39             1              0                           0   
1467   27             1              0                           0   
1468   49             1              0                           0   
1469   34             1              0                           0   

      Department_Research & Development  Department_Sales  Gender_Female  \
0                                     0                 1              1   
1      